In [1]:
import os
import sys
import random

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data

import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

In [2]:
# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter

In [3]:
# 计划使用GPU运行
# 检查是否有可用的GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
batchSize = 64
imageSize = 28      # 输入图像的宽和高
nz = 100            # 隐藏变量的维度
ngf = 28            # 生成器中卷积层的数量
ndf = 28            # 判别器中卷积层的数量
niter = 200         # 训练的轮数
lr = 0.0002         # 学习率
beta1 = 0.5         # Adam优化器的beta1参数
nc = 1              # 输入图像的通道数

In [5]:
os.makedirs("./demo2_outf", exist_ok=True)

In [6]:
# transform配置
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(
            mean=0.5,
            std=0.5,
        )
    ]
)

In [7]:
# 加载mnist数据，同时将数据按照transform的配置预处理
# 判断是什么操作系统，以设置相应的数据集路径
if sys.platform.startswith('win'):
    datasetpath = 'D:\\nndatasets'
else:
    datasetpath = '/home/cxmd/文档/data_for_AI_train/mnist_data'

mnist = dset.MNIST(
    root=datasetpath,
    train=True,
    transform=transform,
    download=True,
)

In [8]:
# 数据集加载器：GAN为无监督机器学习
# 参数dataset是要加载的数据集，这里是预处理后的mnist数据集
# 参数batch_size是每个批次的样本数量
# 参数shuffle为True表示在每个训练周期开始时，对数据进行重新洗牌
data_loader = torch.utils.data.DataLoader(
    dataset = mnist,
    batch_size=batchSize,
    shuffle=True,
)
data_loader

In [9]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(in_channels=nz, out_channels=ngf * 8, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 28 x 28
        )

    def forward(self, input):
        return self.main(input)

In [10]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # 输入是 (nc) x 28 x 28
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 14 x 14
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 7 x 7
            nn.Conv2d(ndf * 2, 1, 7, 1, 0, bias=False),
            nn.Sigmoid()
        )
        
    def forward(self, input):
        return self.main(input).view(-1, 1).squeeze(1)

In [11]:
# 生成实例
netG = Generator().to(device)
netD = Discriminator().to(device)
print(netG)
print(netD)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 224, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(224, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(224, 112, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(112, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(112, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): Tanh()
  )
)
Discriminator(
  (main): Sequential(
    (0): Conv2d(1, 28, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(28, 56, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(56, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(56, 1, kerne

In [ ]:
# 为生成器创建一个SummaryWriter对象
writerG = SummaryWriter('./demo2_outf/Generator')

# 为判别器创建一个SummaryWriter对象
writerD = SummaryWriter('./demo2_outf/Discriminator')

# 生成一些随机输入数据
input_to_netG = torch.rand(batchSize, nz, 1, 1, device=device)
input_to_netD = torch.rand(batchSize, nc, imageSize, imageSize, device=device)

# 将模型的结构写入TensorBoard
writerG.add_graph(netG, input_to_model=input_to_netG)
writerD.add_graph(netD, input_to_model=input_to_netD)

# 关闭writers
writerG.close()
writerD.close()